# Saturation Methods Analysis

**Created by:** Samuel Ropert  
**Creation date:** 12/10/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook shows how periodic examinations affect a pandemy evolution.


## Import Libraries

In [18]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time
import multiprocessing
from joblib import Parallel, delayed

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR2 import SEIR
from Quarantine import Quarantine
from Quarantine import Exams

Linux


In [19]:
def simulate(simulation,j,tsim):
    simulation.integr_sci(0,tsim,0.1)
    return simulation

# Kinetic Saturation

## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

In [21]:
beta = 0.2 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 1000000
# Initial Active Infected 
I0 = 100
I_ac0 = 100

# Immunity Shield
k_R=0

# Kinetic Saturation
k_I = [0,5,10,20,50]

# Quarantine
s1 = Quarantine(0.5)

## Create simulation Object

In [22]:
# Define Simulations
sims = []
for i in range(len(k_I)):        
    sims.append(SEIR(tsim=tsim,alpha=s1.alpha,beta=beta,mu=mu,k_I=k_I[i],k_R=k_R,I0=I0,population=population,expinfection=1,SeroPrevFactor=1))

## Simulate
The different scenarios are simulated in parallel threads 

In [23]:
num_cores = multiprocessing.cpu_count()
simulation = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],0,tsim) for i in range(len(k_I)))
print('simulation finished')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    1.0s remaining:    1.5s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    1.0s remaining:    0.7s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    1.1s finished
simulation finished


# Plots

###  Active infected

In [25]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(k_I)))
for i in range(len(k_I)):
    plt.plot(simulation[i].t,simulation[i].I,label='K_I='+str(k_I[i]), color =  colors[i])    

plt.title('Kinetic Saturation')
plt.legend(loc=0)
plt.show()

# Immunity Shield

In [26]:
beta = 0.2 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 1000000
# Initial Active Infected 
I0 = 100
I_ac0 = 100

# Immunity Shield
k_R = [0,5,10,20,50]

# Kinetic Saturation
k_I = 0

# Quarantine
s1 = Quarantine(0.5)

## Create simulation Object

In [27]:
# Define Simulations
sims = []
for i in range(len(k_R)):        
    sims.append(SEIR(tsim=tsim,alpha=s1.alpha,beta=beta,mu=mu,k_I=k_I,k_R=k_R[i],I0=I0,population=population,expinfection=1,SeroPrevFactor=1))
    

## Simulate
The different scenarios are simulated in parallel threads 

In [28]:
num_cores = multiprocessing.cpu_count()
simulation = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],0,tsim) for i in range(len(k_R)))
print('simulation finished')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.7s remaining:    0.5s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.8s finished
simulation finished


In [29]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(k_R)))
for i in range(len(k_R)):
    plt.plot(simulation[i].t,simulation[i].I,label='K_R='+str(k_R[i]), color =  colors[i])    

plt.title('Immunity Shield')
plt.legend(loc=0)
plt.show()